In [11]:
import csv
import re
import pandas as pd
#None para visualizar todas las columnas, False para la version recortada
pd.set_option("display.max_columns", None)


#Parseo de datos

In [12]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Mineria/'
filename = "dataset_pln_20211019.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Se parsean los datos manualmente y no con pandas,
# porque es mas facil para agregar las cantidades de respuestas de cada tuit
is_first_cell = True
tweets_dicc = {}

with open(drive_path+filename, mode='r') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    if is_first_cell:
      print(f'Column names are {", ".join(row)}')
      is_first_cell = False

    id_str = row["id_str"]
    in_reply_to_status_id = row["in_reply_to_status_id"]
    full_text = row["full_text"]
    
    created_at = row["created_at"]
    in_reply_to_user_id = row["in_reply_to_user_id"]
    user_id = row["user_id"]

    # Eliminar hashtags, menciones y urls
    full_text = re.sub('#\w+ ', '', full_text)
    full_text = re.sub('@\w+ ', '', full_text)
    full_text = re.sub(r'http\S+', '', full_text)
    row["full_text"] =full_text


    # Contar las respuestas y ponerlo en una nueva columna

    # si ya existe un tweet de ese id, es porque se creo' porque tenia una respuesta
    if tweets_dicc.get(id_str):
      tweets_dicc[id_str].update(row)
    # si es la primera vez que lo vimos, creamos el dict por completo
    else:
      tweets_dicc[id_str] = {**{"ans": 0}, **row }

    #si es respuesta de x
    if in_reply_to_status_id:
      #si es la primera respuesta que recibe x
      if not tweets_dicc.get(in_reply_to_status_id,{}).get("ans",0):
        #si todavia no vimos a x
        if not tweets_dicc.get(in_reply_to_status_id, {}).get("id_str"):
          tweets_dicc[in_reply_to_status_id] = {"ans": 1}
        #si ya vimos a x
        else:
          tweets_dicc[in_reply_to_status_id].update({"ans": 1})
      #si x ya tuvo otra respuesta
      else:
        tweets_dicc[in_reply_to_status_id]["ans"] +=1

    # print(created_at, id_str, full_text, in_reply_to_status_id, in_reply_to_user_id, user_id)

Column names are created_at, id_str, full_text, in_reply_to_status_id, in_reply_to_user_id, user_id


In [14]:
#Borrar los tuits que no tienen texto en la bd
#i y j son para contar la cantidad de tuits que hay con y sin texto
dic_ans_num = {}
tw_wo_text = []
# i = 0
# j = 0
for tw_id in tweets_dicc:
  # print(tweets_dicc[tw_id].get("id_str"), tweets_dicc[tw_id].get("ans"))
  # j+=1
  id = tweets_dicc[tw_id].get("id_str")
  ans = tweets_dicc[tw_id].get("ans")
  dic_ans_num[ans] = dic_ans_num.get(ans,0)+1
  
  #hay i (358) tweets de 163441 que tienen respuesta, pero no tienen texto en la bd
  if not id :
    tw_wo_text.append(tw_id)
    # i +=1
    # print(id, tw_id, ans)

# print(len(tweets_dicc))
#borrando del dicc los tw sin texo
for id in tw_wo_text:
  del tweets_dicc[id]
# print(i, j, j-i)
# print(len(tweets_dicc))


In [15]:
#Agregar los tags a los tuits segun su cantidad de respuestas
dic_ans_class = {"0":0,
                 "1":0,
                 "2-3":0,
                 "4-6":0,
                 "7-10":0,
                 "11-20":0,
                 "20+":0}

for tw_id in tweets_dicc:
  ans = tweets_dicc[tw_id].get("ans")
  if ans ==0:
    dic_ans_class["0"]+=1
    tweets_dicc[tw_id]["class"] = "0"
  elif ans <=1:
    dic_ans_class["1"]+=1
    tweets_dicc[tw_id]["class"] = "1"
  elif ans <=3:
    dic_ans_class["2-3"]+=1
    tweets_dicc[tw_id]["class"] = "2-3"
  elif ans <=6:
    dic_ans_class["4-6"]+=1
    tweets_dicc[tw_id]["class"] = "4-6"
  elif ans <=10:
    dic_ans_class["7-10"]+=1
    tweets_dicc[tw_id]["class"] = "7-10"
  elif ans <=20:
    dic_ans_class["11-20"]+=1
    tweets_dicc[tw_id]["class"] = "11-20"
  else:
    dic_ans_class["20+"]+=1
    tweets_dicc[tw_id]["class"] = "20+"


In [16]:
#paso los tuits a un dataframe para dividirlo mas facilmente
tweets_df = pd.DataFrame.from_dict(tweets_dicc, orient='index')
# print(tweets_df)

In [17]:
#Dividir el df en 80 10 10
#training_df tiene el 80%
training_df = tweets_df.sample(frac = 0.80)
#rest tiene el 20%
rest = tweets_df.drop(training_df.index)

#develop tiene el 10% (50% del 20% de rest)
develop_df = rest.sample(frac = 0.50) 

#test tiene el 10% (50% del 20% de rest)
test_df =  rest.drop(develop_df.index)

# print(len(training_df), len(develop_df), len(test_df))
# print(len(training_df)+ len(develop_df)+ len(test_df))

In [19]:
training_df.to_csv(drive_path+'training_df.csv', index=False)
develop_df.to_csv(drive_path+'develop_df.csv', index=False)
test_df.to_csv(drive_path+'test_df.csv', index=False)
